<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c5e52c7d744e7a616a37feaea104081301d1a8e4a3a2a2bda11b91bfa633a4bc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-10 15:25:16
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: -24.16 K (-0.17%)
Current PnL: -29.33 L (-18.95%)
CY Booked + Current PnL: -15.04 L (-9.72%)
-------------------
Total profit:  1.19 L
Total loss:  -30.52 L
-------------------
Total Booked + Current PnL: 11.94 L (9.34%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.28%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.97 L (67.63%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.79,-17.38,21.15,0.09,17610.0,-17513.0,83264.0,93.17,37.0,M-SC,2.35,253.0,-0.99,0.62,9.39,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,2.01,-12.98,14.94,0.02,22694.0,-22659.0,151900.0,131.58,51.0,M-SC,16.60,234.0,-1.00,1.13,32.13,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.65,-2.31,24.94,22.05,23871.0,-2265.0,95715.0,-15.99,55.0,H-SC,6.86,164.0,-0.09,0.71,39.13,XR,ATH,FINANCE
43,ITC,452.00,0.50,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,47.0,X-LC,1.72,5.0,-0.13,1.47,3.96,X40,NTT,FMCG
82,VOLTAS,1530.00,0.49,5.02,13.97,19.69,28130.0,9618.0,201360.0,-5.22,46.0,X-MC,2.57,78.0,0.34,1.50,12.15,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SATIN,274.0,3.40,-21.36,93.08,51.83,202229.0,-59020.0,217264.0,-29.04,38.0,H-SC,1.92,146.0,-0.29,1.62,5.84,XY24,NTT,FINANCE
44,JCHAC,2282.0,3.09,-35.98,56.18,-0.01,45968.0,-45983.0,81822.0,16639.13,24.0,M-SC,4.11,233.0,-1.00,0.61,4.72,OX40N,NTT,AC
36,ICICIPRULI,790.0,3.01,6.50,22.98,30.97,43254.0,11485.0,188223.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.40,19.88,X40,ATH,INSURANCE
30,HATHWAY,31.4,2.56,-29.14,144.55,73.29,111361.0,-31680.0,77040.0,794.93,43.0,H-SC,7.31,172.0,-0.28,0.57,3.88,XR,NTT,ENTERTAINMENT
18,COFFEEDAY,80.0,2.36,-45.05,146.15,35.27,91198.0,-51149.0,62400.0,-57.75,27.0,L-SC,10.22,270.0,-0.56,0.47,52.73,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-8.54,-20.61,53.14,21.58,59123.0,-28881.0,111258.0,-59.96,16.0,X-MC,6.35,56.0,-0.49,0.83,0.00,X40N,ATH,IT
61,ROUTE,2227.21,-2.99,-49.07,231.95,69.05,157183.0,-65300.0,67766.0,-58.67,45.0,H-SC,23.22,140.0,-0.42,0.51,4.51,SR,ATH,IT
74,TITAGARH,1548.00,-2.69,-29.10,105.56,45.74,181241.0,-70482.0,171695.0,-5.57,25.0,H-SC,4.05,174.0,-0.39,1.28,11.67,XY24,NTT,ENGINEERING
57,RAJOOENG,143.10,-2.61,-37.29,122.62,39.61,78820.0,-38220.0,64280.0,-57.57,23.0,H-SC,26.26,135.0,-0.48,0.48,0.00,AR,ATH,MISC
23,DMART,5391.45,-2.41,-9.33,41.30,28.12,78792.0,-19627.0,190780.0,-21.75,30.0,X-LC,4.64,19.0,-0.25,1.42,12.09,X40N,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.52,-6.59,122.58,107.91,169053.0,-9732.0,137912.0,-23.79,39.0,M-SC,12.16,216.0,-0.06,1.03,2.83,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.01,-12.98,14.94,0.02,22694.0,-22659.0,151900.0,131.58,51.0,M-SC,16.60,234.0,-1.00,1.13,32.13,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.79,-17.38,21.15,0.09,17610.0,-17513.0,83264.0,93.17,37.0,M-SC,2.35,253.0,-0.99,0.62,9.39,OX40N,NTT,DURABLES
66,SIS,528.00,-0.38,-24.95,61.57,21.26,51309.0,-27698.0,83334.0,1964.77,45.0,H-SC,4.46,166.0,-0.54,0.62,12.59,OX40N,NTT,MISC
47,KANSAINER,340.00,0.94,-25.24,51.79,13.47,104404.0,-68076.0,201591.0,-69.28,30.0,H-SC,3.66,158.0,-0.65,1.50,2.50,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.16,5.75,62.9,72.27,112712.0,9744.0,179192.0,-9.26,65.0,M-LC,2.09,99.0,0.09,1.34,12.28,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.16,5.75,62.90,72.27,112712.0,9744.0,179192.0,-9.26,65.0,M-LC,2.09,99.0,0.09,1.34,12.28,XR,NTT,IT
38,INDIAMART,4810.62,-0.86,-3.81,114.91,106.72,136323.0,-4701.0,118635.0,-52.34,31.0,H-SC,1.58,139.0,-0.03,0.89,18.68,AR,ATH,MISC
25,FINCABLES,1641.55,1.52,-6.59,122.58,107.91,169053.0,-9732.0,137912.0,-23.79,39.0,M-SC,12.16,216.0,-0.06,1.03,2.83,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.40,-2.57,39.20,35.63,80044.0,-5381.0,204193.0,-16.30,38.0,H-MC,3.16,119.0,-0.07,1.52,13.68,AR,ATH,PHARMA
50,MASFIN,398.61,0.65,-2.31,24.94,22.05,23871.0,-2265.0,95715.0,-15.99,55.0,H-SC,6.86,164.0,-0.09,0.71,39.13,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-8.54,-20.61,53.14,21.58,59123.0,-28881.0,111258.0,-59.96,16.0,X-MC,6.35,56.0,-0.49,0.83,0.00,X40N,ATH,IT
75,TMPV,600.00,-0.45,-18.68,74.85,42.18,168235.0,-51647.0,224763.0,-26.74,24.0,X-LC,5.62,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.71,-21.31,88.65,48.45,139876.0,-42722.0,157784.0,-26.96,25.0,X-MC,11.68,64.0,-0.31,1.18,2.73,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-0.20,-10.46,26.94,13.67,62010.0,-26883.0,230180.0,-17.11,29.0,X-LC,2.79,14.0,-0.43,1.72,8.67,XY25,NTT,FMCG
52,NESTLEIND,1377.00,-0.60,6.54,13.94,21.40,39421.0,17363.0,282789.0,-1.17,30.0,X-LC,5.93,12.0,0.44,2.11,13.62,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,0.47,-21.76,96.20,53.50,227105.0,-65675.0,236076.0,-62.10,33.0,X-MC,1.34,58.0,-0.29,1.76,4.86,XY24,NTT,FMCG
15,CAMS,4762.0,-1.77,-80.71,546.84,24.79,265704.0,-203267.0,48589.0,-81.54,36.0,X-SC,1.48,86.0,-0.77,0.36,18.78,X40N,NTT,MISC
43,ITC,452.0,0.50,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,47.0,X-LC,1.72,5.0,-0.13,1.47,3.96,X40,NTT,FMCG
36,ICICIPRULI,790.0,3.01,6.50,22.98,30.97,43254.0,11485.0,188223.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.40,19.88,X40,ATH,INSURANCE
3,ACC,3906.0,-0.25,-25.17,119.38,64.15,212556.0,-59901.0,178050.0,-56.31,35.0,X-SC,1.99,82.0,-0.28,1.33,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.25,-25.17,119.38,64.15,212556.0,-59901.0,178050.0,-56.31,35.0,X-SC,1.99,82.0,-0.28,1.33,0.0,XY24,BTT,CEMENT
22,DIXON,18931.72,-8.54,-20.61,53.14,21.58,59123.0,-28881.0,111258.0,-59.96,16.0,X-MC,6.35,56.0,-0.49,0.83,0.0,X40N,ATH,IT
31,HAVELLS,2069.16,-2.12,-11.02,48.89,32.48,142679.0,-36154.0,291837.0,-15.47,34.0,X-MC,5.34,67.0,-0.25,2.18,0.0,X40,ATH,ELECTRICAL
53,PAGEIND,51605.08,-0.11,-9.14,38.74,26.07,57637.0,-14960.0,148780.0,-33.02,30.0,X-MC,11.48,55.0,-0.26,1.11,0.0,X40,ATH,APPARELS
75,TMPV,600.00,-0.45,-18.68,74.85,42.18,168235.0,-51647.0,224763.0,-26.74,24.0,X-LC,5.62,3.0,-0.31,1.68,0.0,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.13,-29.87,103.69,42.84,47269.0,-19419.0,45587.0,-54.27,41.0,X-SC,36.81,83.0,-0.41,0.34,3.14,XY24,NTT,MISC
15,CAMS,4762.00,-1.77,-80.71,546.84,24.79,265704.0,-203267.0,48589.0,-81.54,36.0,X-SC,1.48,86.0,-0.77,0.36,18.78,X40N,NTT,MISC
59,RELAXO,1176.00,-1.83,-46.49,184.61,52.31,143409.0,-67478.0,77682.0,-42.99,47.0,X-SC,6.14,91.0,-0.47,0.58,3.98,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.18,-37.26,115.52,35.20,93250.0,-47948.0,80722.0,5.85,37.0,X-SC,15.38,92.0,-0.51,0.60,2.54,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-8.54,-20.61,53.14,21.58,59123.0,-28881.0,111258.0,-59.96,16.0,X-MC,6.35,56.0,-0.49,0.83,0.00,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-0.87,-12.83,36.83,19.28,111072.0,-44376.0,301580.0,-23.95,63.0,X-LC,5.77,1.0,-0.40,2.25,11.49,X40,ATH,IT
41,INFY,2275.00,-0.96,7.81,43.65,54.87,147244.0,24440.0,337328.0,-16.01,59.0,X-LC,3.28,2.0,0.17,2.52,16.74,X40,BTT,IT
75,TMPV,600.00,-0.45,-18.68,74.85,42.18,168235.0,-51647.0,224763.0,-26.74,24.0,X-LC,5.62,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
81,VBL,671.64,0.12,-4.64,42.27,35.67,127317.0,-14642.0,301200.0,-15.92,53.0,X-LC,4.73,4.0,-0.12,2.25,8.58,X40N,ATH,FMCG
43,ITC,452.00,0.50,-1.52,12.14,10.44,23927.0,-3047.0,197091.0,-41.15,47.0,X-LC,1.72,5.0,-0.13,1.47,3.96,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,1.53,-18.31,122.40,81.67,93795.0,-17180.0,76630.0,6883.33,38.0,L-SC,14.26,271.0,-0.18,0.57,48.40,XR,NTT,CERAMICS
50,MASFIN,398.61,0.65,-2.31,24.94,22.05,23871.0,-2265.0,95715.0,-15.99,55.0,H-SC,6.86,164.0,-0.09,0.71,39.13,XR,ATH,FINANCE
13,BSOFT,831.70,-1.77,-23.35,96.71,50.79,104267.0,-32839.0,107814.0,-1.82,64.0,H-SC,5.81,151.0,-0.31,0.80,25.53,XR,ATH,IT
39,INDIGOPNTS,1408.00,2.01,-12.98,14.94,0.02,22694.0,-22659.0,151900.0,131.58,51.0,M-SC,16.60,234.0,-1.00,1.13,32.13,OX40N,NTT,PAINTS
36,ICICIPRULI,790.00,3.01,6.50,22.98,30.97,43254.0,11485.0,188223.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.40,19.88,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-1.77,-23.35,96.71,50.79,104267.0,-32839.0,107814.0,-1.82,64.0,H-SC,5.81,151.0,-0.31,0.80,25.53,XR,ATH,IT
32,HCLTECH,1922.01,0.49,8.10,15.38,24.73,40223.0,19607.0,261531.0,12.76,63.0,X-LC,5.98,7.0,0.49,1.95,24.57,X40,ATH,IT
84,WIPRO,420.00,0.16,5.75,62.90,72.27,112712.0,9744.0,179192.0,-9.26,65.0,M-LC,2.09,99.0,0.09,1.34,12.28,XR,NTT,IT
56,RAJESHEXPO,518.00,0.71,-63.34,172.99,0.07,86986.0,-86893.0,50284.0,1743.43,55.0,L-SC,2.30,268.0,-1.00,0.38,25.06,OX40N,NTT,JEWELLERY
39,INDIGOPNTS,1408.00,2.01,-12.98,14.94,0.02,22694.0,-22659.0,151900.0,131.58,51.0,M-SC,16.60,234.0,-1.00,1.13,32.13,OX40N,NTT,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.73
1,25,45.80
2,50,76.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.00
MC    30.32
LC    24.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.30
X40      22.51
X40N     10.82
XR       10.07
XY25      9.39
AR        9.33
OX40N     7.80
SR        1.03
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.47
X-MC    23.57
X-LC    21.08
M-SC    12.13
X-SC     6.98
H-MC     4.99
L-SC     1.42
M-MC     1.40
M-LC     1.34
H-LC     1.19
L-LC     1.06
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.40,-7.94,42.84
IT,12.71,-18.27,82.21
FINANCE,9.57,-20.86,71.93
ELECTRICAL,6.24,-12.50,53.71
PAINTS,6.13,-12.29,28.96
MISC,6.07,-59.69,125.11
INSURANCE,4.64,-1.45,37.20
PHARMA,4.12,-1.27,33.68
AUTO,3.40,-26.43,80.65


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3242034.0,21
AR,1322825.0,10
XR,1312654.0,13
X40,1022089.0,14
X40N,999295.0,9
OX40N,746310.0,10
XY25,383108.0,6
SR,283261.0,2
MH,84879.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3697209.0,25
M-SC,1432259.0,15
X-MC,1324756.0,16
X-SC,984421.0,8
X-LC,938979.0,11
H-MC,409751.0,3
L-SC,271979.0,3
M-LC,112712.0,1
H-LC,74484.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1289774.0      6
           AR         905444.0      5
M-SC       XY24       817597.0      6
H-SC       XR         785659.0      7
X-SC       X40N       506425.0      3
X-MC       X40        458632.0      7
           XY24       392566.0      3
X-LC       X40        380307.0      5
H-SC       OX40N      348192.0      4
M-SC       OX40N      311132.0      5
X-SC       XY24       294846.0      3
X-MC       X40N       286761.0      4
H-SC       SR         283261.0      2
X-LC       XY24       251132.0      2
H-MC       AR         213632.0      2
X-LC       X40N       206109.0      2
H-MC       XY24       196119.0      1
X-MC       XY25       186797.0      2
L-SC       XR         184993.0      2
X-SC       X40        183150.0      2
M-SC       XR         174265.0      2
           AR         129265.0      2
M-LC       XR         112712.0      1
X-LC       XY25       101431.0      2
L-SC       OX40N       86986.0      1
H-SC       MH          84879.0      1
H-LC       AR          74484.0      1
M-MC       XY25        57055.0      1
L-MC       XR          55025.0      1
L-LC       XY25        37825.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
